In [8]:
import json,os,sys,logging,pprint,re,string

In [88]:
nGramDict=json.load(open('/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/Sample-Dict/mixed-nGramDict_skim_1.json'))

In [90]:
nGramDict['_n']['W9']

{'_n': {'W1': {'_n': {'W1012': {'_r': ['W26']},
    'W1013': {'_r': ['W15']},
    'W1017': {'_r': ['W41']},
    'W10186': {'_r': ['W33']},
    'W10224': {'_r': ['W2508']},
    'W1025': {'_r': ['W942']},
    'W10263': {'_r': ['W26']},
    'W10296': {'_r': ['W571']},
    'W10303': {'_r': ['W33']},
    'W10381': {'_r': ['W26']},
    'W10483': {'_r': ['W4165']},
    'W1055': {'_r': ['W26', 'W22']},
    'W10557': {'_r': ['W346']},
    'W10617': {'_r': ['W33']},
    'W1062': {'_r': ['W26']},
    'W10655': {'_r': ['W33']},
    'W1068': {'_r': ['W26', 'W490']},
    'W1083': {'_r': ['W26']},
    'W1088': {'_r': ['W33']},
    'W1089': {'_r': ['W34616']},
    'W10950': {'_r': ['W26']},
    'W11075': {'_r': ['W182']},
    'W1110': {'_r': ['W51']},
    'W11113': {'_r': ['W26']},
    'W111984': {'_r': ['W26', 'W31']},
    'W112': {'_r': ['W26']},
    'W1126': {'_r': ['W26']},
    'W11284': {'_r': ['W26']},
    'W11294': {'_r': ['W26']},
    'W113': {'_r': ['W13843']},
    'W1146': {'_r': ['W26', 'W3

In [92]:
def cleanInputPredict(test_input):
    cleanedInput=[]
    test_tokens=test_input.strip().lower().split()
    test_input=re.sub('\n+'," ",test_input)
    for item in test_tokens:
        item = item.strip(string.punctuation)
        item = re.sub(r"^.*\d+.*$","<Quantity>",item)
        cleanedInput.append(item)
    return cleanedInput

In [93]:
def predictCore(input_tokens,output_list,out_len=5):
    print (input_tokens)
    if len(input_tokens)<1:
        output_list+=(nGramDict['_r'])
    else:
        # probe if in ngram record
        nGram_flag=True
        i=0
        p_Dict=nGramDict['_n']
        while nGram_flag:
            print(i)
            if i==len(input_tokens):
                break
            if (input_tokens[i] in p_Dict):
                r_wids=p_Dict[input_tokens[i]]['_r']
                if ('_n' in p_Dict[input_tokens[i]]):
                    p_Dict=p_Dict[input_tokens[i]]['_n']
                    i+=1
                else:
                    if i<len(input_tokens)-1:
                        nGram_flag=False
                    break
            else:
                nGram_flag=False
                break
        # if true
        if nGram_flag:
            output_list+=r_wids
            if len(output_list)<out_len:
                del input_tokens[0]
                predictCore(input_tokens,output_list,out_len=out_len)
        # else false
        else:
            del input_tokens[0]
            predictCore(input_tokens,output_list,out_len=out_len)


        
        
        

In [124]:
def predictNgram(test_input,out_len):
    #clean input string
    input_tokens=cleanInputPredict(test_input)
    if len(input_tokens)>nGramDict['_model']:
        input_tokens=input_tokens[-nGramDict['_model']:]
    output_list=[]
    output_wlist=[]
    output=[]
    input_idtokens=[]
    for w in input_tokens:
        w.strip()
        if w in nGramDict['_word2id']['_word']:
            input_idtokens.append(nGramDict['_word2id']['_word'][w])
        else:
            input_idtokens.append('NA')
    predictCore(input_idtokens,output_list,out_len)
    for wid in output_list:
        output_wlist.append(nGramDict['_id2word']['_id'][wid])
    eosflag=True
#     print(output_wlist)
    for word in output_wlist:
        if word=='<EOS>':
            if eosflag:
                output.append(',')
                output.append('.')
            eosflag=False
        elif word=="<Quantity>":
            continue
        else:
            output.append(word)
        #dedupe
        output_dp=[output[0]]
        if len(output)>1:
            for i in range(1,len(output)):
                if output[i] not in output[:i]:
                    output_dp.append(output[i])
        if len(output_dp)>out_len:
            output_dp=output_dp[0:out_len]
    return output_dp

In [135]:
test_input="I like how the same people are in almost all of Adam Sandler's"

In [136]:
predictNgram(test_input,8)

['W13', 'W33', 'W5847', 'W131331']
0
1
2
['W33', 'W5847', 'W131331']
0
1
['W5847', 'W131331']
0
1
['W131331']
0
[]


['the', 'to', 'and', 'a']